<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 6: Derivación e integración numérica

Este tema lo construyo en parte basado en https://brianheinold.net/notes/An_Intuitive_Guide_to_Numerical_Methods_Heinold.pdf, que proporciona un enfoque bastante más práctico del tema.

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np
from tqdm import tqdm

from scipy.linalg import orth

from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
from anmi.T5 import polinomio_newton, roots_chebyshev

In [ ]:
from anmi.genericas import norma_p_func, norma_inf_func, matriz_inversa
from anmi.T5 import polinomio_lagrange, polinomio_newton, error_lagrange, error_maximo_estocastico, roots_chebyshev
from anmi.T5 import aitken_neville, interpolacion_hermite, polinomio_generico, esplines

In [ ]:
x, y, z, a, lambda_, h = symbols('x'), symbols('y'), symbols('z'), symbols('a'), symbols('lambda'), symbols('h')

# Derivación numérica

## El problema de la aritmética finita en el cálculo de derivadas

El problema principal del cálculo de derivadas está en que el denominador de la derivada se hace muy pequeño:
$f'(x) = \frac{f(x+h) - f(x)}{h}$

En ese sentido, si empezamos a usar valores de $h$ muy pequeños, podemos acabar teniendo valores de derivada alejados de la realidad. Como ejemplo, vamos a evaluar la derivada de $2x$ en $x=3$, empleando diferentes valores de $h$:

In [ ]:
for n in range(23):
    print(n, (((x+h) ** 2 - x**2) / (h)).subs(h, 10**(-n)).subs(x, 3))

Vemos que para valores de h de $10^{-13}$ para abajo, las derivadas se nos van de madre. Esto es problemático porque para derivadas más complejas el error aumenta y no nos podremos fiar directamente de los resultados. Para mitigar estos problemas en estas secciones trabajaremos con diferentes métodos de aproximación de derivadas.

## Fórmulas de derivación numérica
En esta sección vamos a derivar diferentes métodos de cálculo de derivadas.


## Creando el polinomio de Newton
Una de las opciones para calcular la derivada $n$-ésima es generar un polinomio de grado $n$ y aplicar su derivada $n$ veces. Para ello podemos emplear el método de Newton.

In [ ]:
help(polinomio_newton)

In [ ]:
for i in range(1):
    print(i)

In [ ]:
help(derivacion_polinomio_newton)

In [ ]:
def derivacion_polinomio_newton(f, x0, var=symbols('x'), grado=1, h=0.01):
    x_vals = [x0 + i * h for i in range(grado + 1)]
    y_vals = [f.subs(var, (x0 + i * h)) for i in range(grado + 1)]
    
    # Con esto creamos el polinomio
    p, m = polinomio_newton(x_vals, y_vals, var)
    
    # Ahora lo diferenciamos
    p_diff = p
    for i in range(grado):
        p_diff = p_diff.diff(var)
    
    
    
    return p_diff, p, m

#### EJEMPLO 32

In [ ]:
p_diff, p, m =  derivacion_polinomio_newton(f=sqrt(x), x0=1, var=symbols('x'), grado=3, h=0.05)

In [ ]:
expand(p)

In [ ]:
N(p_diff)

In [ ]:
N(m)

Ahora vamos a jugar con otros valores

In [ ]:
p_diff, p, m =  derivacion_polinomio_newton(f=sqrt(x), x0=1, var=symbols('x'), grado=3, h=0.01)
N(p_diff)

In [ ]:
p_diff, p, m =  derivacion_polinomio_newton(f=sqrt(x), x0=1, var=symbols('x'), grado=3, h=0.001)
N(p_diff)

In [ ]:
p_diff, p, m =  derivacion_polinomio_newton(f=sqrt(x), x0=1, var=symbols('x'), grado=3, h=0.0001)
N(p_diff)

Vemos que con $h$ más pequeños, el valor se asemeja más al verdadero de 0.375

In [ ]:
p_diff, p, m =  derivacion_polinomio_newton(f=sqrt(x), x0=1, var=symbols('x'), grado=3, h=0.0000001)
N(p_diff)

In [ ]:
p_diff, p, m =  derivacion_polinomio_newton(f=sqrt(x), x0=1, var=symbols('x'), grado=3, h=0.000000001)
N(p_diff)

Sin embargo, como antes, si tomamos $h$ extremadamente pequeños, el cálculo se nos va de madre.

## Cálculo de error en derivadas
Muchas veces nos interesa poder hallar el error de aproximación de la derivada, para ver hasta qué punto nos parece una aproximación adecuada.
Recordemos que la expansión de Taylor de $f(x)$ en $x=a$ es:
$$f(x) = f(a) + f'(a)(x-a) + \frac{f^{(2)}(a)}{2!}(x-a)^2 + \frac{f^{(3)}(a)}{3!}(x-a)^3 + \cdots$$
Si sustituimos $x$ for $x+h$ y tomando $a$ como $x-h$, la fórmula anterior queda como:
$$f(x+h) = f(x) + f'(x)h + \frac{f^{(2)}(x)}{2!}h^2 + \frac{f^{(3)}(x)}{3!}h^3 + \cdots$$
Resolviendo para $f'(x)$ tenemos:
$$ f'(x) = \frac{f(x+h) - f(x)}{h} - \frac{f^{(2)}(x)}{2!}h - \frac{f^{(3)}(x)}{3!}h^2 - \cdots$$

Luego el error de aproximación pertenecería a los términos de derivada segunda y superiores. Para simplificar, nos quedamos solo con el término de derivada segunda y se cumple que, para un $c \in [x, x+h]$
$$f'(x) = \frac{f(x+h) - f(x)}{h} - \frac{f^{(2)}(c)}{2!}h$$
De modo que podemos acotar el error.

Esta estrategia de cálculo de error la emplearemos después en otros métodos.

## Derivación con polinomios de Lagrange
Otro método es construyendo polinomios de Lagrange. Recordemos que los polinomios vienen dados por la fórmula:
$$p_n(x) = \sum_{i=0}^n l_i(x)f(x_i)$$
Si aplicamos una derivación a $p_n(x)$ en $\bar{x}_0$ obtenemos el siguiente polinomio:
$$D(f)(\bar{x_0}) = \sum_{i = 0}^n l'_i(\bar{x}_0)f(x_i)$$
$\bar{x_0}$ es un punto cualquiera del intervalo, y puede o no coincidir con alguno de los nodos.

Recordemos que la construcción de $p(x)$ viene dada por la base $\{1, x, \cdots, x^n\}$ y cada termino es $x^j = \sum_{i=0}^n l_i(x)x^j$, de modo que, matricialmente, $p(x)$ se construye como:
$$\begin{bmatrix}
1 & 1 & \cdots & 1 \\
x_0 & x_1 & \cdots & x_n\\
\vdots & \vdots & \ddots & \vdots\\
x_0^n & x_1^n & \cdots & x_n^n\\
\end{bmatrix}
\begin{bmatrix}
l_0\\
l_1\\
\vdots\\
l_n\\
\end{bmatrix} = 
\begin{bmatrix}
1\\
x\\
\vdots\\
x^n
\end{bmatrix}
$$

Si derivamos $r$ veces en $\bar{x}_0$, acabamos con el sistema:
$$\begin{bmatrix}
1 & 1 & \cdots & 1 \\
x_0 & x_1 & \cdots & x_n\\
\vdots & \vdots & \ddots & \vdots\\
x_0^n & x_1^n & \cdots & x_n^n\\
\end{bmatrix}
\begin{bmatrix}
l^{(r)}_0=a_0\\
l^{(r)}_1=a_1\\
\vdots\\
l^{(r)}_n=a_n\\
\end{bmatrix} = 
\begin{bmatrix}
0\\
0\\
\vdots\\
(n)(n-1)\cdots(n-r)\bar{x}_0^{n-r}
\end{bmatrix}
$$

In [ ]:
help(polinomio_lagrange_derivada)

TODO: implementar la función de error

In [ ]:
def polinomio_lagrange_derivada(f, x0, x_vals=None, var=symbols('x'), grado=1, I=[0, 1]):
    ff = Function('f')
    
    if x_vals is None:
        x_vals = np.linspace(I[0], I[1], grado + 1)
    assert len(x_vals) == grado + 1
    
    G = ones(len(x_vals), len(x_vals))
    for row in range(1, len(x_vals)):
        for col in range(len(x_vals)):
            G[row, col] = x_vals[col] ** row
            
    x_mat = Matrix([var ** i for i in range(grado + 1)])
    for _ in range(grado):
        x_mat = x_mat.diff(var)
    
    # Con esto creamos el vector de valores de A
    a = matriz_inversa(G) * x_mat
    
    p, val = S(0), S(0)
    for i in range(grado + 1):
        p += a[i] * ff(x_vals[i])
        val += a[i] * f.subs(var, x_vals[i])
    
    return p, val
    

In [ ]:
polinomio_lagrange_derivada(f=x**3, x0=2, x_vals=[1, 2, 3], var=symbols('x'), grado=2)

## Método de extrapolación de Richardson
Para explicar el método me voy a basar en la explicación de Wikipedia, que generaliza mejor, con cambios de notación. https://en.wikipedia.org/wiki/Richardson_extrapolation

Supongamos que tenemos una función $N(h)$, que la empleamos como aproximación a la derivada en $h=0$, $N(0)$. La función $N(h)$ va a tener una expansión del error del siguiente modo:
$$N(h) = N(0) + a_0h^{k_0} + a_1h^{k_1} + a_2h^{k_2} + \cdots$$

En primer lugar, tomamos un parametro $r$, de modo que podemos sustituir $h$ por $rh$ del siguiente modo:
$$N(rh) = N(0) + a_0(rh)^{k_0} + a_1(rh)^{k_1} + a_2(rh)^{k_2} + \cdots$$

Si multiplicamos por $(r)^{k_0}$ y restamos tenemos que:
$$r^{k_0}N(h) =  N(0) + a_0r^{k_0}h^{k_0} + a_1r^{k_0}h^{k_1} + a_2r^{k_0}h^{k_2} + \cdots$$

$$r^{k_0}N(h) -  N(rh)=  N(0) + (r^{k_0} - r^{k_1})a_1h^{k_1} + (r^{k_0} - r^{k_2})a_2h^{k_2} + \cdots$$

Luego
$$N_1(h) = \frac{r^{k_0}N(h) -  N(rh)}{r^{k_0} - 1}= N(0) + \frac{r^{k_0} - r^{k_1}}{r^{k_0} - 1}a_1h^{k_1} + \frac{r^{k_0} - r^{k_2}}{r^{k_0} - 1}a_2h^{k_2} + \cdots$$

Vemos que ahora tenemos una nueva función, $N_1(h)$, que depende de $a_1h^{k_1} + a_2h^{k_2} + \cdots$, luego hemos reducido su error, ya que $\frac{r^{k_0} - r^{k_1}}{r^{k_0} - 1}$ tiene menos influencia que el orden de $h$. Con iteraciones sucesivas podemos hacer este error tan pequeño como veamos, dentro de la capacidad de error aritmética de $r^nh$ para el ordenador.

Generalizando, la fórmula final es:
$$N_a(r^bh) = \frac{r^{m+a-1}N(r^bh) - N(r^{b+1}h)}{r^{m+a-1} - 1}$$

El valor $m=k_0$ depende de $N(h)$. Si, por ejemplo, $N(h) = \frac{f(x+h) - f(x-h)}{2h}$, considerando los desarrollos de Taylor:
$$f(x+h) = f(x) + f'(x)fh + \frac{1}{2}f''(x)h^2 + \frac{1}{3!}f'''(x)h^3 + \frac{1}{4!}f''''(x)h^4 + \cdots$$
$$f(x-h) = f(x) - f'(x)fh + \frac{1}{2}f''(x)h^2 - \frac{1}{3!}f'''(x)h^3 + \frac{1}{4!}f''''(x)h^4 + \cdots$$

Se tiene que $N(h) = f'(x) + \frac{h^2}{3!}f'''(x) + \frac{h^4}{5!}f^{(5)}(x) +\cdots$, luego $m=2$

Para simplificar los cálulos, se hace un cálculo tabular de todas estas sucesiones. Para ello se hacen los cálculos de la siguiente manera:
$$
\begin{matrix}
h & N(h) & N_1(h) & N_2(h) & N_3(h) \\
rh & N(rh) & N_1(rh) & N_2(rh) & \cdots \\
r^2h & N(r^2h) & N_1(r^2h) & \cdots & \cdots \\
r^3h & N(r^3h) & \cdots & \cdots & \cdots \\
\vdots & \vdots & \vdots & \vdots & \ddots \\
\end{matrix}
$$

In [ ]:
def derivada_richardson(f, x0=0, r=0.5, h=0.5, x_var=symbols('x'), h_var=symbols('h'), Nf=None, m=None, grado=3):
    if Nf is None:
        Nf = (f.subs(x_var, x0 + h_var) - f.subs(x_var, x0 - h_var))/(2 * h_var)
        m = 2
        
    matriz_r = zeros(grado, grado)
    
    # primero llenamos la matriz con los valores N(rih)
    for row in range(grado):
        rih = r ** row * h
        matriz_r[row, 0] = nsimplify(Nf.subs(h_var, rih),tolerance=1e-10,rational=True)
        
    for col in range(1, grado):
        for row in range(grado - col):
            r_exp = m + col - 1
            matriz_r[row, col] = nsimplify((r**r_exp * matriz_r[row, col - 1] - matriz_r[row+1, col - 1])/(r**r_exp - 1), rational=True)
            
    
    return Nf, matriz_r
    

#### EJERCICIO 60

In [ ]:
f = sin(2*pi*x)
x0, r, h = 0, 0.5, 0.5

Nf, matriz_r = derivada_richardson(f=f, x0=x0, r=r, h=h, grado=3)
Nf

In [ ]:
matriz_r

#### EJEMPLO de HTML

In [ ]:
f = -cos(x)
x0, r, h = 1, 0.5, 1

Nf, matriz_r = derivada_richardson(f=f, x0=x0, r=r, h=h, grado=6)
N(matriz_r)

# Integración numérica

## Cuadratura basada en la interporlación

El método de cuadratura basado en intepolación está basado en el polinomio de Legendre. La metodología es la misma que para la derivada, solo que sustituyendo la derivada con la integral.

$$\begin{bmatrix}
1 & 1 & \cdots & 1 \\
x_0 & x_1 & \cdots & x_n\\
\vdots & \vdots & \ddots & \vdots\\
x_0^n & x_1^n & \cdots & x_n^n\\
\end{bmatrix}
\begin{bmatrix}
l^{(r)}_0=a_0\\
l^{(r)}_1=a_1\\
\vdots\\
l^{(r)}_n=a_n\\
\end{bmatrix} = 
\begin{bmatrix}
\int_a^b 1 dx = b-a\\
\int_a^b x dx =  \frac{b^2-a^2}{2}\\
\vdots\\
\int_a^b x^n dx =  \frac{b^{n+1}-a^{n+1}}{n+1}
\end{bmatrix}
$$

Para el método de newton, el sistema de matrices a resolver es:
$$\begin{bmatrix}
1 & 1 & \cdots & 1 \\
0 & \omega_1(x_1) & \cdots & \omega_1(x_n)\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & \omega_n(x_n)\\
\end{bmatrix}
\begin{bmatrix}
l_0\\
l_1\\
\vdots\\
l_n\\
\end{bmatrix} = 
\begin{bmatrix}
1\\
\omega_1\\
\vdots\\
\omega_n
\end{bmatrix}
$$

Donde $\omega_i(x) = (x-x_0)(x-x_1)\cdots(x-x_{i-1})$

Si integramos el rhs tenemos el sistema:
$$\begin{bmatrix}
1 & 1 & \cdots & 1 \\
0 & \omega_1(x_1) & \cdots & \omega_1(x_n)\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & \omega_n(x_n)\\
\end{bmatrix}
\begin{bmatrix}
l_0\\
l_1\\
\vdots\\
l_n\\
\end{bmatrix} = 
\begin{bmatrix}
\int_a^b dx\\
\int_a^b \omega_1(x) dx\\
\vdots\\
\int_a^b \omega_n(x) dx
\end{bmatrix}
$$

In [ ]:
help(polinomio_lagrange_integracion)

TODO: implementar la función de error

In [ ]:
def polinomio_lagrange_integracion(f=x, x0=None, x_vals=None, grado=3, var=symbols('x'), I=[0, 1]):
    ff = Function('f')
    
    if x_vals is None:
        x_vals = np.linspace(I[0], I[1], grado)
    
    G = ones(len(x_vals), len(x_vals))
    for row in range(1, len(x_vals)):
        for col in range(len(x_vals)):
            G[row, col] = x_vals[col] ** row
            
    x_mat = nsimplify(Matrix([(I[1] ** (i + 1) - I[0] ** (i + 1))/(i + 1) for i in range(len(x_vals))]), rational=True)

    # Con esto creamos el vector de valores de A
    a = nsimplify(matriz_inversa(G) * x_mat, rational=True)
    
    p, val = S(0), S(0)
    for i in range(len(x_vals)):
        p += a[i] * ff(x_vals[i])
        val += a[i] * f.subs(var, x_vals[i])
    
    return p, val

def polinomio_newton_integracion(f=x, x0=None, x_vals=None, grado=3, var=symbols('x'), I=[0, 1]):
    ff = Function('f')
    
    if x_vals is None:
        x_vals = np.linspace(I[0], I[1], grado)
    
    # Creamos la lista de omegas, que tienen la expresión [w_1, w_2, w_3, etc]
    lista_w = [1, (var - x_vals[0])]
    for x_val in x_vals[1:]:
        lista_w.append(lista_w[-1] * (var - x_val))
    
    G = zeros(len(x_vals), len(x_vals))
    x_mat = zeros(len(x_vals), 1)
    for row in range(len(x_vals)):
        for col in range(row, len(x_vals)):
            if row == 0:
                G[row, col] = 1
            else:
                G[row, col] = lista_w[row].subs(var, x_vals[col])
    
        x_mat[row, 0] = integrate(lista_w[row], (x, I[0], I[1]))
    
    # Con esto creamos el vector de valores de A
    a = matriz_inversa(G) * x_mat
    
    p, val = S(0), S(0)
    for i in range(len(x_vals)):
        p += a[i] * ff(x_vals[i])
        val += a[i] * f.subs(var, x_vals[i])
    
    return p, val

#### EJERCICIO 61

In [ ]:
p, v = polinomio_lagrange_integracion(x_vals=[1, 2, 3], I=[0, 4])
p

In [ ]:
p, v = polinomio_newton_integracion(x_vals=[1, 2, 3], I=[0, 4])
p

#### EJERCICIO 62

In [ ]:
p, v = polinomio_lagrange_integracion(x_vals=roots_chebyshev(2, I=[-1, 1]), I=[-1, 1])
p

## Estimación del error de cuadratura

El error de cuadratura viene definido como 
$$e=\int_a^bf(x)dx - Q(f)$$

Y se tiene que
$$|e| \le \frac{\max_{\xi \in [a,b]} f^{(n+1)}(\xi)}{(n+1)!} \int_a^b|\omega_{n+1}(x)| $$

In [ ]:
def integral_positiva_newton(omega, I, var, x_vals):
    # Esta función la creo para que calcule la integral del valor absoluto. Como las integrales son de la forma (x-x0)(x-x1)(x-x2)(x-x3)...
    # con xi diferentes y en orden creciente, la función va a tomar un valor negativo si el número de términos es impar, y positivo si par
    # para a < x_0, y de ahí va a ir alternando signos. Como pueden surgir casos diferentes (que I[0] sea x_vals[0]) y la casuistica cambia
    # para cada x_i, x_i+1 tomaremos la integral y evaluaremos el signo, para ponerlo a la integral de nuevo.
    integral = S(0)
    
    x_vals_I = [I[0]] + x_vals + [I[1]]
    for i in range(len(x_vals_I) - 1):
        a, b = x_vals_I[i], x_vals_I[i+1]
        integral_i = Integral(omega, (var, a, b)).doit()
        
        if integral_i > 0:
            signo = 1
        elif integral_i < 0:
            signo = -1
        else: # a = b y la integral es cero
            continue
        integral += signo * integral_i
    
    return integral
        
def error_cuadratura_integral(f, var=symbols('x'), x_vals=None, I=[0,1], grado=2):
    if x_vals is None:
        x_vals = np.linspace(I[0], I[1], grado)
    
    # Hallamos el primer término, el del máximo de la derivada
    diff_f = f
    for _ in range(len(x_vals)):
        diff_f = diff_f.diff(var)

    max_diff_f = nsimplify(maximum(diff_f, var, Interval(I[0], I[1])) / factorial(len(x_vals)), rational=True)
    
    # Ahora hallamos el de la integral
    omega = S(1)
    for x_val in x_vals:
        omega *= (var - x_val)
    
    int_omega = nsimplify(integral_positiva_newton(omega, I, var, x_vals), rational=True)
    
    e = max_diff_f * int_omega
    
    return e, max_diff_f, int_omega

In [ ]:
#### EJERCICIO 62

In [ ]:
e, max_diff_f, int_omega = error_cuadratura_integral(f=sin(pi*x), x_vals=roots_chebyshev(2, I=[-1, 1]), I=[-1, 1])
e

## Fórmulas cerradas de Newton-Cotes

Una de las maneras de aproximar la integral de una función es dividirla en $n$ nodos y aproximar la integral mediante una función de tipo $\int_a^bf(x)dx \sim A_0f(a=x_0) + A_1f(x_1) + A_2f(x_2) + \cdots + A_nf(x_n)$.

Según el grado del polinomio de aproximación que queramos, tenemos las siguientes fórmulas:
* grado 0 (extremo izquierdo): $\int_a^b f(x) dx \sim (b-a)f(a)$
* grado 1 (trapecios): $\int_a^b f(x) dx \sim \frac{b-a}{2}(f(a) + f(b))$
* grado 2 (Simpson): $\int_a^b f(x) dx \sim \frac{b-a}{6}(f(a) + 4f(\frac{a+b}{2}) + f(b))$

Que, quitando parafernalia, buscan una manera de aproximar con polinomios de diferentes grados la función entre $a$ y $b$. Obviamente, si la función es complicada, la aproximación no va a ser tan buena, pero para ello veremos métodos de como aplicarlo por partes.

![irudia.png](../img/newton_cotes.png)

¿Y cómo se puede hallar la aproximación para grado $n$? Para ello empleamos la cuadratura basada en la interpolación, que hemos visto antes:

$$\begin{bmatrix}
1 & 1 & \cdots & 1 \\
x_0 & x_1 & \cdots & x_n\\
\vdots & \vdots & \ddots & \vdots\\
x_0^n & x_1^n & \cdots & x_n^n\\
\end{bmatrix}
\begin{bmatrix}
\alpha_0\\
\alpha_1\\
\vdots\\
\alpha_n\\
\end{bmatrix} = 
\begin{bmatrix}
\int_a^b 1 dx = b-a\\
\int_a^b x dx =  \frac{b^2-a^2}{2}\\
\vdots\\
\int_a^b x^n dx =  \frac{b^{n+1}-a^{n+1}}{n+1}
\end{bmatrix}
$$

De hecho, si aplicamos esto para grados 0, 1 y 2, obtenemos los $\alpha_i$ de los casos de extremo izquierdo ($\alpha_0 = 1$), trapecios ($\alpha_0 = \alpha_1 = 1/2$) y Simpson ($\alpha_0 = \alpha_2 = 1/6$, $\alpha_2 = 2/3$).

In [ ]:
def formulas_newton_cotes(f, orden=1, x_vals=None,  y_vals=None, I=[0, 1], var=symbols('x')):
    ff = Function('f')
    
    if x_vals is None:
        x_vals = np.linspace(I[0], I[1], orden + 1)
    if y_vals is None:
        y_vals = [f.subs(var, i) for i in x_vals]
    
    G = ones(len(x_vals), len(x_vals))
    for row in range(1, len(x_vals)):
        for col in range(len(x_vals)):
            G[row, col] = x_vals[col] ** row
    
    x_mat = nsimplify(Matrix([(I[1] ** (i + 1) - I[0] ** (i + 1))/(i + 1) for i in range(len(x_vals))]), rational=True)

    # Con esto creamos el vector de valores de A
    a = nsimplify(matriz_inversa(G) * x_mat, rational=True)
    
    Q, val = S(0), S(0)
    for i in range(len(x_vals)):
        Q += a[i] * ff(nsimplify(x_vals[i], rational=True))
        val += nsimplify(a[i] * f.subs(var, x_vals[i]), rational=True)
    
    return Q, val

#### EJERCICIO 63


In [ ]:
Q, val = formulas_newton_cotes(1/(1+x**2), orden=0, I=[-5, 5])
Q

In [ ]:
Q, val = formulas_newton_cotes(1/(1+x**2), orden=1, I=[-5, 5])
Q

In [ ]:
Q, val = formulas_newton_cotes(1/(1+x**2), orden=2, I=[-5, 5])
Q

In [ ]:
Q, val = formulas_newton_cotes(1/(1+x**2), orden=3, I=[-5, 5])
Q

In [ ]:
Q, val = formulas_newton_cotes(1/(1+x**2), orden=4, I=[-5, 5])
Q

In [ ]:
for i in range(1, 15):
    Q, val = formulas_newton_cotes(1/(1+x**2), orden=i, I=[-5, 5])
    print(i, val)

### Error de la fórmula de cuadratura del trapecio y de newton

Para una aproximación de grado $n$, el error de aproximación viene definido por:
$$e_n(f) = \int_a^bf[a=x_0, x_1, x_2, \cdots, x_n=b]\omega_n(x)dx$$

Para el caso del trapecio (grado 1) el error es: 
$$e_1(f) = \frac{\max_{\eta \in [a,b]} |f''(\eta)|}{12}(b-a)^3$$

Para el caso del trapecio (grado 2) el error es: 
$$e_2(f) = \frac{\max_{\eta \in [a,b]} |f^{(4)}(\eta)|}{90}\left(\frac{b-a}{2}\right)^5$$

Para $\eta \in [a, b]$

In [ ]:
def error_trapecio(f, var=symbols('x'), I=[0,1]):
    diff_f = f
    for _ in range(2):
        diff_f = diff_f.diff(var)
        
    max_diff_f = nsimplify(maximum(diff_f, var, Interval(I[0], I[1])), rational=True)
                           
    factor_division = (I[1] - I[0]) ** 3 / 12
    
    e = max_diff_f * factor_division   
                           
    return e, max_diff_f, factor_division        


def error_simpson(f, var=symbols('x'), I=[0,1]):
    diff_f = f
    for _ in range(4):
        diff_f = diff_f.diff(var)
        
    max_diff_f = nsimplify(maximum(diff_f, var, Interval(I[0], I[1])), rational=True)
                           
    factor_division = ((I[1] - I[0])/2) ** 5 / 90
    
    e = max_diff_f * factor_division   
                           
    return e, max_diff_f, factor_division  

In [ ]:
e, max_diff, int_omega = error_trapecio(1/(1+x**2), I=[-5, 5])
N(e)

In [ ]:
e, max_diff, int_omega = error_cuadratura_integral(1/(1+x**2), x_vals=[-5, 5], I=[-5, 5])
N(e)

In [ ]:
e, max_diff, int_omega = error_simpson(1/(1+x**2), I=[-5, 5])
N(e)

In [ ]:
e, max_diff, int_omega = error_cuadratura_integral(1/(1+x**2), x_vals=[-5, 0, 5], I=[-5, 5])
N(e)

## Cuadratura compuesta

Uno de los problemas clásicos de emplear los polinomios de cuadratura para aproximar las integrales es que, para funciones complejas, con máximos y mínimos, la aproximación deja de ser buena para grados altos. Cojamos por ejemplo, $f(x) = x + sin^2(x)$ en $x \in [0, 2\pi]$


In [ ]:
from sympy.plotting import plot

p1 = plot(1/(1+x**2), (x, -4, 4), ylim=[0,1], axis_center=(0,0))

In [ ]:
N(Integral(1/(1+x**2), (x, -4, 4)).doit())

In [ ]:
for i in range(1, 15):
    Q, val = formulas_newton_cotes(1/(1+x**2), orden=i, I=[-4, 4])
    print(i, val)

Vemos que los valores de la integral se van para $n$ altos. Esto tiene sentido en base a lo que hemos visto del fenómeno de Runge y las aproximaciones de grado alto de ciertas funciones, que resultan en polinomios que pueden interpolar bien, pero ser poco efectivos fuera de los nodos.

Para evitar saltos del estilo, podemos calcular en vez de un polinomio de cuadratura en $[a,b]$, calcular $m$ polinomios en el intervalo de $[a,b]$ divido $m$ veces: $[a = x_0, x_1,\cdots, x_{m-1}, x_m=b]$, donde $h = x_i - x_{i-1} = x_{i+1} - x_{i} = \frac{b-a}{m}$. Así, por ejemplo, la cuadratura de trapecios compuesta resultaría en la fórmula:
$$Q_{1,m}(f) = \sum_{i = 0}^{m-1} \frac{h}{2}(f(x_i) + f(x_{i+1})$$
El error de la fórmula es el mismo que el de $Q_1$, solo que propagado a lo largo de $m$ intervalos:
$$E_{1,m}(f) = - \sum_{i = 0}^{m-1}\frac{f''(\eta_i)}{12}h^3$$
Del teorema del valor intermedio podemos simplificar la expresión, teniendo en mente que existe un $\eta \in [a,b]$ tal que:
$$E_{1,m}(f) = - \frac{f''(\eta)}{12}mh^3 = - \frac{f''(\eta)}{12}(b-a)h^2$$


Para la cuadratura de Simpson se procede igual:
$$Q_{2,m} = \sum_{i = 0}^{m-1} \frac{h}{6}(f(x_{2i}) + 4f(x_{2i+1}) + f(x_{2(i+1)}))$$
Aqui el intervalo, aunque está dividido en $m$ partes, para calcular la cuadratura por intervalo, se subdivide en 2 partes, de ahí el $2m$.

El error de la cuadratura de Simpson es:
$$E_{2,m}(f) = - \sum_{i = 0}^{m-1}\frac{f^{(4)}(\eta_i)}{90}\left(\frac{h}{2}\right)^5  \rightarrow - \frac{f^{(4)}(\eta)}{90}m\left(\frac{h}{2}\right)^5 = 
- \frac{f^{(4)}(\eta)}{180}(b-a)\left(\frac{h}{2}\right)^4$$

In [ ]:
def formulas_newton_cotes_m(f, orden=1, m=10, I=[0, 1], var=symbols('x')):
    lista_vals, lista_Q = [], []
    lista_m = np.linspace(I[0], I[1], m + 1)
    
    for m_i in range(m):
        m0, mf = lista_m[m_i], lista_m[m_i + 1]
        Q, val = formulas_newton_cotes(f, orden=orden, I=[m0, mf], var=var)
        lista_Q.append(Q)
        lista_vals.append(val)
    
    Q = sum(lista_Q)
    val = sum(lista_vals)
    
    return Q, val

#### EJEMPLO 34

In [ ]:
Q_12, val_12 = formulas_newton_cotes_m(f=x**4, orden=1, m=2, I=[0, 1], var=symbols('x'))
Q_12

In [ ]:
val_12

In [ ]:
Q_14, val_14 = formulas_newton_cotes_m(f=x**4, orden=1, m=4, I=[0, 1], var=symbols('x'))
Q_14

In [ ]:
val_14

In [ ]:
Q_22, val_22 = formulas_newton_cotes_m(f=x**4, orden=2, m=2, I=[0, 1], var=symbols('x'))
Q_22

In [ ]:
val_22

In [ ]:
def error_trapecio_m(f, var=symbols('x'), I=[0,1], m=2):
    diff_f = f
    for _ in range(2):
        diff_f = diff_f.diff(var)
        
    max_diff_f = nsimplify(maximum(diff_f, var, Interval(I[0], I[1])), rational=True)
                           
    factor_division = (I[1] - I[0]) ** 3 / (12 * (m ** 2))
    
    e = max_diff_f * factor_division   
                           
    return e, max_diff_f, factor_division        


def error_simpson_m(f, var=symbols('x'), I=[0,1], m=2):
    diff_f = f
    for _ in range(4):
        diff_f = diff_f.diff(var)
        
    max_diff_f = nsimplify(maximum(diff_f, var, Interval(I[0], I[1])), rational=True)
                           
    factor_division = (I[1] - I[0]) ** 5 / (90 * (m ** 4) * (2 ** 5))
    
    e = max_diff_f * factor_division   
                           
    return e, max_diff_f, factor_division  

#### EJERCICIO 64
Vamos a resolver el ejercicio iterativamente. Vamos a ver para qué $m$ conseguimos un error menor a 0.0001.

In [ ]:
e, max_diff_f, factor_division = error_trapecio_m(f, I=I, m=symbols('m'), var=x)

In [ ]:
e

In [ ]:
solve_univariate_inequality(e < 0.0001, symbols('m'), False, Interval(0, S.Infinity))

In [ ]:
f = 1/x
I = [1, 3]

for i in range(100, 120):
    e, max_diff_f, factor_division = error_trapecio_m(f, I=I, m=i, var=x)
    print(i, e)

Vemos como para $m=116$ el error es menor a $10^{-4}$. Vamos a calcular el valor de $\ln 3$ para ese $m$

In [ ]:
Q, val = formulas_newton_cotes_m(f=1/x, orden=1, m=116, I=[1, 3], var=symbols('x'))

In [ ]:
N(val)

In [ ]:
N(log(3))

#### EJERCICIO 65

En este caso tenemos que hallar $m$ para resolver $\int_0^1 e^xdx$ con una exactitud de $10^{-5}$. Como tenemos programado para $m$ en lugar de $h$, lo hacemos así.

In [ ]:
f, I = E ** x, [0, 1]

e, max_diff_f, factor_division = error_simpson_m(f=f, I=I, m=symbols('m'), var=x)

In [ ]:
N(solve_univariate_inequality(e < 0.00001, symbols('m'), False, Interval(0, S.Infinity)))

Vemos que $m > 3.11$, luego $m = 4$, tal y como lo proponen en el ejercicio.

In [ ]:
Q, val = formulas_newton_cotes_m(f=f, orden=2, m=4, I=I, var=symbols('x'))
N(val)

In [ ]:
Q

Calculamos el valor de la integral. Sabemos que $\int_0^1 e^x = e |_0^1 = e - 1$

In [ ]:
N(E - 1)

## Fórmulas de Gauss
Recordemos la resolución de cuadratura por la matriz de Vandermonde

$$\begin{bmatrix}
1 & 1 & \cdots & 1 \\
x_0 & x_1 & \cdots & x_n\\
\vdots & \vdots & \ddots & \vdots\\
x_0^n & x_1^n & \cdots & x_n^n\\
\end{bmatrix}
\begin{bmatrix}
\alpha_0\\
\alpha_1\\
\vdots\\
\alpha_n\\
\end{bmatrix} = 
\begin{bmatrix}
\int_a^b 1 dx = b-a\\
\int_a^b x dx =  \frac{b^2-a^2}{2}\\
\vdots\\
\int_a^b x^n dx =  \frac{b^{n+1}-a^{n+1}}{n+1}
\end{bmatrix}
$$

En este caso, estamos estableciendo los nodos de manera equidistante, o arbitrariamente. En el mismo sentido de la aproximación de los polinomios, a la hora de resolver la cuadratura podemos escoger los valores $x_i$ de manera que la resolución de la cuadratura sea mejor. En este caso, los polinomios que establecen son los de Chebyshev y los de Legendre. 
Por tanto, el proceso es el mismo que el de la cuadratura al uso, pero sustituyendo los valores de x por el de las raices del polinomio de Chebyshev/Legendre del grado deseado.

Como los polinomios están descritos en el intervalo $[-1, 1]$, si tenemos un problema en un intervalo $I = [a,b]$ podemos hacer un cambio de variable a $[-1, 1]$, una vez obtenidas las variables para la cuadratura, revertir el cambio de variable. Este cambio de variable **también cambia el valor de $\alpha$!!!**

Si $x = \frac{b-a}{2} t + \frac{b+a}{2}$ tenemos que $$\hat{\alpha}_i = \int_{a}^{b} \hat{l}_i dx = \int_{-1}^{1} l_i(t) dt = \frac{b-a}{2} \alpha_i$$

In [ ]:
def formulas_gauss(f, I=[-1, 1], var=symbols('x'), grado=3, modo='legendre'):
    ff = Function('f')

    if modo=='legendre':
        x_vals = sorted(solve(legendre(grado, var)))
    elif modo=='chebyshev':
        x_vals = sorted(solve(chebyshevt(grado, var)))
    
    G = ones(len(x_vals), len(x_vals))
    for row in range(1, len(x_vals)):
        for col in range(len(x_vals)):
            G[row, col] = x_vals[col] ** row
    
    x_mat = nsimplify(Matrix([((1) ** (i + 1) - (-1) ** (i + 1))/(i + 1) for i in range(len(x_vals))]), rational=True)

    # Con esto creamos el vector de valores de A
    a = nsimplify((I[1] - I[0])/2 * (matriz_inversa(G) * x_mat), rational=True)
    
    # Aquí efectuamos el cambio de variable:
    cambio_var = (I[1] - I[0])/2 * var + (I[1] + I[0])/2
    Q, val = S(0), S(0)
    for i in range(len(x_vals)):
        Q += a[i] * ff( nsimplify(cambio_var.subs(var, x_vals[i]), rational=True))
        val += nsimplify(a[i] * f.subs(var, cambio_var.subs(var, x_vals[i])), rational=True)
        
    return Q, val

#### EJERCICIO 66

In [ ]:
Q, val = formulas_gauss(f=E**(-x**2), I=[0.2, 1.5], var=symbols('x'), grado=3, modo='legendre')
Q

In [ ]:
N(val)

#### EJERCICIO 67

In [ ]:
f = x ** symbols('n')
I = [-1, 1]

Q_leg, val_leg = formulas_gauss(f=f, I=I, var=symbols('x'), grado=2, modo='legendre')
Q_leg

In [ ]:
val_leg

In [ ]:
Q_chev, val_chev = formulas_gauss(f=f, I=I, var=symbols('x'), grado=2, modo='chebyshev')
Q_chev

In [ ]:
val_chev